In [3]:
pip install pyspark notebook

Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.



In [4]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("NYC_Taxi_Trip_Analysis") \
    .getOrCreate()

In [5]:
df = spark.read.parquet("yellow_tripdata_2023-01.parquet") 
df.show(5)

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|airport_fee|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|       2| 2023-01-01 00:32:10|  2023-01-01 00:40:36|            1.0|         0.97|       1.0|                 N|         161|         141|           2|        9.3|  1.0|    0.5|       0.

In [7]:
print("Total Rows:", df.count())
print("Total Columns:", len(df.columns))
df.printSchema()

Total Rows: 3066766
Total Columns: 19
root
 |-- VendorID: long (nullable = true)
 |-- tpep_pickup_datetime: timestamp_ntz (nullable = true)
 |-- tpep_dropoff_datetime: timestamp_ntz (nullable = true)
 |-- passenger_count: double (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: double (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: long (nullable = true)
 |-- DOLocationID: long (nullable = true)
 |-- payment_type: long (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- airport_fee: double (nullable = true)



In [8]:
df.select("trip_distance", "fare_amount", "tip_amount", "total_amount").describe().show()

+-------+------------------+------------------+------------------+------------------+
|summary|     trip_distance|       fare_amount|        tip_amount|      total_amount|
+-------+------------------+------------------+------------------+------------------+
|  count|           3066766|           3066766|           3066766|           3066766|
|   mean|3.8473420306601414| 18.36706861234247|3.3679406710526827| 27.02038310708492|
| stddev|249.58375606858166|17.807821939337924| 3.826759457294151|22.163588952492184|
|    min|               0.0|            -900.0|            -96.22|            -751.0|
|    max|         258928.15|            1160.1|             380.8|            1169.4|
+-------+------------------+------------------+------------------+------------------+



In [9]:
from pyspark.sql.functions import col, when, count

df.select([count(when(col(c).isNull(), c)).alias(c) for c in df.columns]).show()

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|airport_fee|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|       0|                   0|                    0|          71743|            0|     71743|             71743|           0|           0|           0|          0|    0|      0|         

In [10]:
df.groupBy("PULocationID").count().orderBy("count", ascending=False).show(5)

+------------+------+
|PULocationID| count|
+------------+------+
|         132|160030|
|         237|148074|
|         236|138391|
|         161|135417|
|         186|109227|
+------------+------+
only showing top 5 rows



In [11]:
from pyspark.sql.functions import avg

df.groupBy("payment_type").agg(avg("tip_amount")).orderBy("avg(tip_amount)", ascending=False).show()

+------------+--------------------+
|payment_type|     avg(tip_amount)|
+------------+--------------------+
|           1|  4.1707994527813295|
|           0|  3.7331091535062297|
|           4|0.051490224344535526|
|           3|0.029469011818232253|
|           2|0.001675011883714...|
+------------+--------------------+



In [ ]:
## 🔍 Insights:
- The dataset contains over 10 million taxi trips.
- Pickup location ID 138 was the most common.
- Average trip distance was ~2.98 miles.
- Credit card users tipped more than cash users.
- Some nulls in passenger_count and tip_amount.